In [ ]:
import os, shutil, glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL,
                                   fname="flower_photos.tgz",
                                   extract=True)

class Image_handling():
    def plot_image(x_arr):
        fig, ax = plt.subplots(10,10, figsize=(10,10))
        ax =plt.axes()
        ax.imshow
        plt.tight_layout()
        plt.show()

